# Transfer learning with Custom Data with VGG16


Import pre-trained VGG16 model

In [1]:
from keras.applications.vgg16 import VGG16
model = VGG16()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

Import packages

In [5]:
import numpy as np
import time
# from vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
import os

c:\users\infoe\anaconda3\envs\mldl\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Access the data directory

In [7]:
data_path = r'./training-data/'
data = os.listdir(data_path)
print(data)

['cats', 'dogs', 'horses']


In [34]:
img_data_list = []

for dataset in data:
    img_list = os.listdir(data_path+'/'+dataset)
    print('loaded images' + '{}\n'.format(dataset))
    for img in img_list:
        img_path = data_path+'/'+ dataset +'/'+ img
        img = image.load_img(img_path, target_size=(224,224))
        x = image.img_to_array(img)        
        x = preprocess_input(x)
        print('Input image shape', x.shape)
        img_data_list.append(x)

loaded imagescats

Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (22

Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input im

Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)
Input image shape (224, 224, 3)


In [37]:
img_data = np.array(img_data_list)
print(img_data.shape)

(606, 224, 224, 3)


In [42]:
# prepare the data set
num_classes = 3
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,), dtype='int64')

labels[0:202] = 0
labels[202:404]= 1
labels[404:606]=2

names=['cats','dogs','horses']

In [45]:
# Transform label to one-hot vector
Y = np_utils.to_categorical(labels, num_classes)

In [46]:
# suffle data
x,y = shuffle(img_data, Y, random_state=2)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state = 2)

In [52]:
image_input = Input(shape = (224,224, 3))
model = VGG16(input_tensor = image_input, weights = 'imagenet')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### Replace last layer

In [54]:
last_layer = model.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input,out)
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [70]:
for layer in custom_vgg_model.layers[:-1]:
    layer.trainable = False
    
print(custom_vgg_model.layers[3].trainable)

False


In [73]:
custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
t = time.time()
hist = custom_vgg_model.fit(X_train,y_train, batch_size = 32 , epochs=30, validation_data=(X_test, y_test))
print('Training time: %s' %(t-time.time()))
(loss, accuracy) = custom_vgg_model.evaluate(X_test, y_test, batch_size = 10, verbose=1)

print('loss={:.4f}, accuracy: {:.4f}%'.format(loss, accuracy * 100))

Train on 484 samples, validate on 122 samples
Epoch 1/30
484/484 [==============================] - 5s 10ms/step - loss: 0.1134 - acc: 0.9690 - val_loss: 0.0465 - val_acc: 0.9672
Epoch 2/30
484/484 [==============================] - 3s 6ms/step - loss: 0.0031 - acc: 1.0000 - val_loss: 0.0513 - val_acc: 0.9754
Epoch 3/30
484/484 [==============================] - 3s 6ms/step - loss: 0.0019 - acc: 1.0000 - val_loss: 0.0486 - val_acc: 0.9754
Epoch 4/30
484/484 [==============================] - 3s 6ms/step - loss: 7.9255e-04 - acc: 1.0000 - val_loss: 0.0475 - val_acc: 0.9672
Epoch 5/30
484/484 [==============================] - 3s 6ms/step - loss: 5.0972e-04 - acc: 1.0000 - val_loss: 0.0520 - val_acc: 0.9754
Epoch 6/30
484/484 [==============================] - 3s 6ms/step - loss: 2.8934e-04 - acc: 1.0000 - val_loss: 0.0477 - val_acc: 0.9754
Epoch 7/30
484/484 [==============================] - 3s 6ms/step - loss: 1.7601e-04 - acc: 1.0000 - val_loss: 0.1123 - val_acc: 0.9754
Epoch 8/30
48

### Replace last 3 layer

In [ ]:
image_input = Input(shape=(224,224,3))

model =  VGG16(input_tensor=image_input, include_top=True, weights='imagenet')
model.summary()

In [74]:
last_layer= model.get_layer('block5_pool').output
x = Flatten(name='flatten')(last_layer)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
out = Dense(num_classes, activation='softmax', name='output')(x)
custom_vgg_model2 = Model(image_input, out)
custom_vgg_model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [75]:
for layer in custom_vgg_model2.layers[:-3]:
    layer.trainable = False
    
custom_vgg_model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

t = time.time()
hist = custom_vgg_model2.fit(X_train, y_train, batch_size=32, epochs=30, validation_data=(X_test, y_test))
print('Training time: %s' % (t-time.time()))
(loss, accuracy) = custom_vgg_model2.evaluate(X_test, y_test, batch_size = 10, verbose = 1)

print( 'loss = {:.4f}, accuracy: {:.4f}%'.format(loss,accuracy*100))

Train on 484 samples, validate on 122 samples
Epoch 1/30
484/484 [==============================] - 3s 7ms/step - loss: 7.7365 - acc: 0.5000 - val_loss: 7.4270 - val_acc: 0.5328
Epoch 2/30
484/484 [==============================] - 3s 6ms/step - loss: 6.2096 - acc: 0.6074 - val_loss: 8.0271 - val_acc: 0.5000
Epoch 3/30
484/484 [==============================] - 3s 6ms/step - loss: 5.3091 - acc: 0.6591 - val_loss: 1.6365 - val_acc: 0.8770
Epoch 4/30
484/484 [==============================] - 3s 6ms/step - loss: 1.3524 - acc: 0.9029 - val_loss: 2.2027 - val_acc: 0.8443
Epoch 5/30
484/484 [==============================] - 3s 6ms/step - loss: 0.8809 - acc: 0.9380 - val_loss: 1.0046 - val_acc: 0.9180
Epoch 6/30
484/484 [==============================] - 3s 6ms/step - loss: 0.2846 - acc: 0.9752 - val_loss: 0.6584 - val_acc: 0.9508
Epoch 7/30
484/484 [==============================] - 3s 6ms/step - loss: 0.4746 - acc: 0.9649 - val_loss: 1.0038 - val_acc: 0.9344
Epoch 8/30
484/484 [==========